# PROJETO FINAL MÓDULO II
---

In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\luizf\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
import requests
import zipfile
import io
import pandas as pd

In [10]:
url = 'https://download.inep.gov.br/microdados/microdados_enem_2013.zip'
response = requests.get(url, verify= False)
response.raise_for_status()

with zipfile.ZipFile(io.BytesIO(response.content)) as zip:
    zip.printdir()

ConnectionError: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory'))

In [2]:
# Seleção de colunas que serão importadas do dataset
colunas = ['TP_FAIXA_ETARIA', 
            'TP_SEXO',
            'TP_NACIONALIDADE',
            'TP_ST_CONCLUSAO',
            'TP_ESTADO_CIVIL', 
            'TP_COR_RACA',
            'SG_UF_PROVA',
            'TP_PRESENCA_CN',
            'TP_PRESENCA_MT',
            'SG_UF_PROVA', 
            'NU_NOTA_CN', 
            'NU_NOTA_CH', 
            'NU_NOTA_LC', 
            'NU_NOTA_MT', 
            'NU_NOTA_REDACAO',
            'TP_STATUS_REDACAO', 
            'Q003', 
            'Q011', 
            'Q017', 
            'Q022']



In [3]:
# Leitura do arquivo csv com as colunas estabelecidas anteriormente
enem_2013_df = pd.read_csv('MICRODADOS_ENEM_2013.csv', encoding= 'latin-1', sep=';', usecols=colunas)

In [4]:
#Renomeação das colunas para melhor legibilidade
enem_2013_df.rename(columns={'Q003':'RENDA_FAMILIAR', 
                             'Q011':'TEM_AUTOMOVEL',
                             'Q017':'TEM_INTERNET',
                             'Q022':'TRABALHA',
                             'SG_UF_PROVA':'UF_PROVA',
                             'ST_CONCLUSAO':'STATUS_CONCLUSAO_EM',
                             'TP_NACIONALIDADE':'NACIONALIDADE',
                             'TP_PRESENCA_CN':'PRESENCA_DIA_1',
                             'TP_PRESENCA_MT':'PRESENCA_DIA_2',
                             'NU_NOTA_LC':'NOTA_LC',
                             'NU_NOTA_CN':'NOTA_CN',
                             'NU_NOTA_CH':'NOTA_CH',
                             'NU_NOTA_MT':'NOTA_MT',
                             'NU_NOTA_REDACAO':'NOTA_REDACAO',
                             'TP_FAIXA_ETARIA':'FAIXA_ETARIA',
                             'TP_SEXO':'SEXO', 
                             'TP_COR_RACA':'COR', 
                             'TP_STATUS_REDACAO':'STATUS_REDACAO'}, inplace=True)

In [5]:
# Alteração dos valores de colunas específicas

enem_2013_df['STATUS_REDACAO'] = enem_2013_df['STATUS_REDACAO'].replace({1:'em branco',
                                                           2:'anulada', 
                                                           3:'fuga tema', 4:'n atende tipo txt',
                                                           5:'txt insuficiente', 6:'ausente', 
                                                           7:'txt conforme requisitos',
                                                           9:'fere DH', 
                                                           10:'cópia txt motivador',
                                                           11:'parte txt desconetada do tema'})

enem_2013_df['COR'] = enem_2013_df['COR'].replace({0:'Não delcarado', 
                                                   1:'Branca', 
                                                   2:'Preta', 
                                                   3:'Parda',
                                                   4:'Amarela', 
                                                   5:'Indígena'})

enem_2013_df['RENDA_FAMILIAR'] = enem_2013_df['RENDA_FAMILIAR'].replace({'A': 0, 
                                                       'B': 1,
                                                       'C': 2,
                                                       'D': 3,
                                                       'E': 4,
                                                       'F': 5,
                                                       'G': 6,
                                                       'H': 7,
                                                       'I': 8,
                                                       'J': 9,
                                                       'K': 10,
                                                       'L': 11,
                                                       'M': 12,
                                                       'N': 13,
                                                       'O': 14,
                                                       'P': 15,
                                                       'Q': 16})

enem_2013_df['TEM_AUTOMOVEL'] = enem_2013_df['TEM_AUTOMOVEL'].replace({'A': 1,
                                                                       'B': 1,
                                                                       'C': 1,
                                                                       'D': 0})

enem_2013_df['TEM_INTERNET'] = enem_2013_df['TEM_INTERNET'].replace({'A': 1,
                                                                     'B': 1,
                                                                     'C': 1,
                                                                     'D': 0})

C:\Users\luizf\AppData\Local\Temp\ipykernel_23140\3876699750.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  enem_2013_df['RENDA_FAMILIAR'] = enem_2013_df['RENDA_FAMILIAR'].replace({'A': 0,
C:\Users\luizf\AppData\Local\Temp\ipykernel_23140\3876699750.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  enem_2013_df['TEM_AUTOMOVEL'] = enem_2013_df['TEM_AUTOMOVEL'].replace({'A': 1,
C:\Users\luizf\AppData\Local\Temp\ipykernel_23140\3876699750.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be remov

### **Análise Automóvel**
- Relação entre posse de automóvel e presença nos dias de prova
---

In [6]:
# Novo dataframe apenas com as coluans necessárias para a análise
df_automovel = enem_2013_df[['TEM_AUTOMOVEL','RENDA_FAMILIAR','PRESENCA_DIA_1','PRESENCA_DIA_2','COR','NOTA_CN','NOTA_CH','NOTA_LC','NOTA_MT','NOTA_REDACAO']]

In [7]:
# Criação de uma nova coluna com o valor final de cada candidato registrado na base
df_automovel['NOTA_FINAL'] = (df_automovel['NOTA_CH'] + df_automovel['NOTA_CN'] + df_automovel['NOTA_LC'] + df_automovel['NOTA_MT'] + df_automovel['NOTA_REDACAO']) / 5

C:\Users\luizf\AppData\Local\Temp\ipykernel_23140\4177428784.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_automovel['NOTA_FINAL'] = (df_automovel['NOTA_CH'] + df_automovel['NOTA_CN'] + df_automovel['NOTA_LC'] + df_automovel['NOTA_MT'] + df_automovel['NOTA_REDACAO']) / 5


In [8]:
df_automovel

,TEM_AUTOMOVEL,RENDA_FAMILIAR,PRESENCA_DIA_1,PRESENCA_DIA_2,COR,NOTA_CN,NOTA_CH,NOTA_LC,NOTA_MT,NOTA_REDACAO,NOTA_FINAL
0,0,3,1,1,Branca,474.9,429.3,520.2,530.2,320,454.92
1,0,1,1,1,Parda,426.9,495.0,485.5,352.5,180,387.98
2,0,1,1,1,Parda,387.9,443.5,422.4,386.3,560,440.02
3,1,5,0,0,Branca,NaN,NaN,NaN,NaN,0,NaN
4,1,2,1,1,Branca,460.4,490.9,519.3,526.8,560,511.48
...,...,...,...,...,...,...,...,...,...,...,...
7173558,0,0,0,0,Preta,NaN,NaN,NaN,NaN,0,NaN
7173559,0,2,1,1,Branca,424.2,362.4,331.5,387.4,0,301.10
7173560,0,1,1,0,Branca,438.1,484.4,NaN,NaN,0,NaN
7173561,1,8,1,1,Branca,508.6,362.0,399.3,477.0,400,429.38


In [9]:
# Tratamento de dados NaN atribuídos a candidatos faltantes. O valor foi substituído pela nota 0.
df_automovel = df_automovel.fillna(0)
df_automovel

,TEM_AUTOMOVEL,RENDA_FAMILIAR,PRESENCA_DIA_1,PRESENCA_DIA_2,COR,NOTA_CN,NOTA_CH,NOTA_LC,NOTA_MT,NOTA_REDACAO,NOTA_FINAL
0,0,3,1,1,Branca,474.9,429.3,520.2,530.2,320,454.92
1,0,1,1,1,Parda,426.9,495.0,485.5,352.5,180,387.98
2,0,1,1,1,Parda,387.9,443.5,422.4,386.3,560,440.02
3,1,5,0,0,Branca,0.0,0.0,0.0,0.0,0,0.00
4,1,2,1,1,Branca,460.4,490.9,519.3,526.8,560,511.48
...,...,...,...,...,...,...,...,...,...,...,...
7173558,0,0,0,0,Preta,0.0,0.0,0.0,0.0,0,0.00
7173559,0,2,1,1,Branca,424.2,362.4,331.5,387.4,0,301.10
7173560,0,1,1,0,Branca,438.1,484.4,0.0,0.0,0,0.00
7173561,1,8,1,1,Branca,508.6,362.0,399.3,477.0,400,429.38


In [11]:
# Candidatos com automóvel e que compareceram aos dois dias de prova
df_automovel[(df_automovel['TEM_AUTOMOVEL'] == 1) & (df_automovel['PRESENCA_DIA_1'] == 1) & (df_automovel['PRESENCA_DIA_2'] == 1)]

,TEM_AUTOMOVEL,RENDA_FAMILIAR,PRESENCA_DIA_1,PRESENCA_DIA_2,COR,NOTA_CN,NOTA_CH,NOTA_LC,NOTA_MT,NOTA_REDACAO,NOTA_FINAL
4,1,2,1,1,Branca,460.4,490.9,519.3,526.8,560,511.48
10,1,6,1,1,Branca,508.2,453.5,566.6,610.5,460,519.76
12,1,1,1,1,Parda,346.5,478.7,423.6,449.7,480,435.70
14,1,4,1,1,Parda,456.5,490.6,471.7,359.4,460,447.64
27,1,4,1,1,Branca,481.0,445.1,434.3,376.8,560,459.44
...,...,...,...,...,...,...,...,...,...,...,...
7173538,1,4,1,1,Parda,411.2,487.3,443.6,485.8,400,445.58
7173547,1,5,1,1,Parda,623.6,589.9,546.4,766.9,560,617.36
7173551,1,4,1,1,Branca,381.1,534.2,416.0,552.7,520,480.80
7173557,1,1,1,1,Preta,340.8,457.5,452.8,472.2,560,456.66


In [19]:
# Definição do do total de candidatos com automóvel e que compareceram aos dois dias de prova, e o total de candidatos registrados
automovel_presentes = df_automovel[(df_automovel['TEM_AUTOMOVEL'] == 1) & (df_automovel['PRESENCA_DIA_1'] == 1) & (df_automovel['PRESENCA_DIA_2'] == 1)].shape[0]
total_candidatos = df_automovel.shape[0]

# Porcentagem de candidatos que respeitam a condição, em relação ao total de registros
porcentagem_presentes_automovel = (automovel_presentes/total_candidatos) * 100
porcentagem_presentes_automovel

32.831328030436204

In [37]:
# Cálculo de evasão parcial de candidatos com automóvel
auto_presente_dia1 = df_automovel[(df_automovel['TEM_AUTOMOVEL'] == 1) & (df_automovel['PRESENCA_DIA_1'] == 1) & (df_automovel['PRESENCA_DIA_2'] == 0)].shape[0]
auto_presente_dia2 = df_automovel[(df_automovel['TEM_AUTOMOVEL'] == 1) & (df_automovel['PRESENCA_DIA_1'] == 0) & (df_automovel['PRESENCA_DIA_2'] == 1)].shape[0]
evasao_parcial_automovel = ((auto_presente_dia1 + auto_presente_dia2) / total_candidatos)*100
evasao_parcial_automovel

1.175664031946189

In [36]:
# Cálculo de evasão parcial de candidatos sem automóvel
sem_auto_presente_dia1 = df_automovel[(df_automovel['TEM_AUTOMOVEL'] == 0) & (df_automovel['PRESENCA_DIA_1'] == 1) & (df_automovel['PRESENCA_DIA_2'] == 0)].shape[0]
sem_auto_presente_dia2 = df_automovel[(df_automovel['TEM_AUTOMOVEL'] == 0) & (df_automovel['PRESENCA_DIA_1'] == 0) & (df_automovel['PRESENCA_DIA_2'] == 1)].shape[0]
evasao_parcial_sem_automovel = ((sem_auto_presente_dia1 + sem_auto_presente_dia2) / total_candidatos)*100
evasao_parcial_sem_automovel

1.7264084806950186

In [20]:
# Candidatos sem automoóvel e que compareceram aos dois dias de prova
df_automovel[(df_automovel['TEM_AUTOMOVEL'] == 0) & (df_automovel['PRESENCA_DIA_1'] == 1) & (df_automovel['PRESENCA_DIA_2'] == 1)]

,TEM_AUTOMOVEL,RENDA_FAMILIAR,PRESENCA_DIA_1,PRESENCA_DIA_2,COR,NOTA_CN,NOTA_CH,NOTA_LC,NOTA_MT,NOTA_REDACAO,NOTA_FINAL
0,0,3,1,1,Branca,474.9,429.3,520.2,530.2,320,454.92
1,0,1,1,1,Parda,426.9,495.0,485.5,352.5,180,387.98
2,0,1,1,1,Parda,387.9,443.5,422.4,386.3,560,440.02
6,0,1,1,1,Branca,437.0,516.2,378.4,486.9,260,415.70
8,0,1,1,1,Preta,372.2,530.9,411.1,525.2,300,427.88
...,...,...,...,...,...,...,...,...,...,...,...
7173552,0,2,1,1,Parda,535.9,583.7,546.7,651.5,680,599.56
7173553,0,1,1,1,Preta,460.0,596.2,480.8,631.0,420,517.60
7173554,0,3,1,1,Branca,629.6,687.8,583.5,677.8,560,627.74
7173559,0,2,1,1,Branca,424.2,362.4,331.5,387.4,0,301.10


In [21]:
# Definição do total de candidatos sem automóvel e que compareceram aos dois dias de prova, e o total de candidatos registrados
sem_automovel_presentes = df_automovel[(df_automovel['TEM_AUTOMOVEL'] == 0) & (df_automovel['PRESENCA_DIA_1'] == 1) & (df_automovel['PRESENCA_DIA_2'] == 1)].shape[0]

# Porcentagem de candidatos que respeitam a condição, em relação ao total de registros
porcentagem_presentes_sem_automovel = (sem_automovel_presentes/total_candidatos) * 100
porcentagem_presentes_sem_automovel

36.97964317034645

In [26]:
# Total de candidatos com completa ausência na prova
ausentes = df_automovel[(df_automovel['PRESENCA_DIA_1'] == 0) & (df_automovel['PRESENCA_DIA_2'] == 0)].shape[0]

In [28]:
# Cálculo da taxa de evasão do ENEM 2013
tx_evasao = (ausentes/total_candidatos) * 100
tx_evasao

27.1708075889206

In [30]:
# Cálculo da porcentagem de participantes com automóvel
tem_automovel = df_automovel[(df_automovel['TEM_AUTOMOVEL'] == 1)].shape[0]

tx_automovel = (tem_automovel/total_candidatos) * 100
tx_automovel

44.12002794148459

### Conclusão
- **44.12%** dos participantes tem automóvel. 
- **27.17%** dos candidatos não estiveram presentes em nenhum dos dois dias de prova
- **32.83%** dos participantes que alegaram ter ao menos 1 automóvel estiveram presentes nos dois dias de prova
- **36.97%** dos participantes que alegaram não ter nenhum automóvel estiveram presentes nos dois dias de prova
- **1.17%** de evasão parcial de candidatos com automóvel
- **1.79%** de evasão parcial de candidatos sem automóvel


- Embora o automóvel seja um facilitador para chegar ao local da prova, mais pessoas sem automóvel compareceram aos dois dias de prova (**36.97%**) do que pessoas com automóvel (**32.83%**). Uma possibildiade é de que muitos candidatos se preocupam menos com a hora por terem o automóvel como meio de transporte, mas acabam se atrasando e perdendo a prova enquanto pessoas que não tem automóvel se organizam e chegam mais cedo ao local da prova, reduzindo o número de atrasos.

---

### **Análise Automóvel**
- Relação entre posse de automóvel e notas
---

In [ ]:
# Contagem de candidatos por faixa de renda
df_automovel['RENDA_FAMILIAR'].value_counts()

In [45]:
# Quantidade de pessoas com automóvel por faixa de renda
contagem_automovel_renda = df_automovel[df_automovel['TEM_AUTOMOVEL'] == 1].groupby('RENDA_FAMILIAR').size()
contagem_automovel_renda

RENDA_FAMILIAR
0      21542
1     399262
2     591930
3     415809
4     294691
5     330616
6     272911
7     225834
8     144575
9      75444
10     82006
11     55139
12     41410
13     62775
14     48771
15     35301
16     66962
dtype: int64

In [43]:
# Quantidade de pessoas sem automóvel por faixa de renda
contagem_sem_automovel_renda = df_automovel[df_automovel['TEM_AUTOMOVEL'] == 0].groupby('RENDA_FAMILIAR').size()
contagem_sem_automovel_renda

RENDA_FAMILIAR
0      115673
1     1742420
2     1093782
3      454496
4      219491
5      176670
6      100910
7       53039
8       22092
9        8874
10       7183
11       3878
12       2403
13       2918
14       1723
15       1143
16       1890
dtype: int64

In [65]:
""" Visto que a faixa de renda 2 foi a faixa de renda com a maior concentração de pessoas com automóvel 
e a segunda maior sem automóvel, ela foi utilizada como base. Como o número de pessoas sem automóvel é maior,
serão utilizadas a média e mediana para efeito comparativo."""

# DataFrame de pessoas da faixa de renda 2 que tem carro e compareceram aos dois dias de prova.
renda_2_automovel = df_automovel[(df_automovel['RENDA_FAMILIAR'] == 2) & (df_automovel['PRESENCA_DIA_1'] == 1) & (df_automovel['PRESENCA_DIA_2'] == 1) & (df_automovel['TEM_AUTOMOVEL'] == 1)]


# Média final desse grupo
media_renda_autmovel = renda_2_automovel['NOTA_FINAL'].mean()
# Mediana desse grupo
mediana_renda_autmovel = renda_2_automovel['NOTA_FINAL'].median()

print(f"""Média final: {media_renda_autmovel}
Mediana: {mediana_renda_autmovel}""")


Média final: 488.60845622136884
Mediana: 484.96000000000004


In [68]:
# DataFrame de pessoas da faixa de renda 2 que não tem automoóvel e compareceram aos dois dias de prova.
renda_2_sem_automovel = df_automovel[(df_automovel['RENDA_FAMILIAR'] == 2) & (df_automovel['PRESENCA_DIA_1'] == 1) & (df_automovel['PRESENCA_DIA_2'] == 1) & (df_automovel['TEM_AUTOMOVEL'] == 0)]


# Média final desse grupo
media_renda_sem_automovel = renda_2_sem_automovel['NOTA_FINAL'].mean()
# Mediana desse grupo
mediana_renda_sem_automovel = renda_2_sem_automovel['NOTA_FINAL'].median()

print(f"""Média final: {media_renda_sem_automovel}
Mediana: {mediana_renda_sem_automovel}""")

Média final: 487.0689724921976
Mediana: 483.9


### Conclusão

- Por se tratar de um grupo bem reduzido em relação ao todo e não trabalhar com todas as faixas de renda, a análise possivelmente não é confiável. No entanto a amostra indica que, a princípio, ter ou não um automóvel não tem grandes interferências no resultado final da prova.
---

### Análise Estrangeiros
- Estrangeiros tem dificuldade com provas relacionadas à língua portuguesa (Linguagens e Códigos | Redação)
---

In [25]:
# DataFrame com os estrangeiros que participaram da edição 2013
enem_2013_df[enem_2013_df['NACIONALIDADE'] == 3]

,FAIXA_ETARIA,SEXO,NACIONALIDADE,TP_ST_CONCLUSAO,TP_ESTADO_CIVIL,COR,UF_PROVA,PRESENCA_DIA_1,PRESENCA_DIA_2,NOTA_CN,NOTA_CH,NOTA_LC,NOTA_MT,STATUS_REDACAO,NOTA_REDACAO,RENDA_FAMILIAR,TEM_AUTOMOVEL,TEM_INTERNET,TRABALHA
555,12,M,3,1,0,Branca,AM,1,1,363.6,495.4,454.7,417.5,txt conforme requisitos,160,0,0,0,B
1130,13,F,3,3,0,Branca,MS,1,1,408.1,611.0,542.1,469.5,txt conforme requisitos,520,2,1,1,B
2527,6,F,3,1,0,Branca,RJ,1,1,491.5,551.0,525.3,657.6,txt conforme requisitos,600,13,0,1,B
3218,5,M,3,1,0,Parda,RO,0,0,NaN,NaN,NaN,NaN,ausente,0,1,0,0,C
3358,4,M,3,1,0,Parda,SP,1,1,522.7,466.1,556.9,521.6,txt conforme requisitos,340,4,1,1,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7165069,6,F,3,1,0,Branca,BA,1,1,432.4,541.1,550.4,494.8,txt conforme requisitos,480,4,1,0,B
7165307,4,F,3,2,0,Branca,RS,1,1,462.1,402.9,498.3,567.1,txt conforme requisitos,440,2,1,1,C
7165999,11,F,3,1,1,Branca,RJ,1,1,588.0,730.2,690.5,584.4,txt conforme requisitos,780,11,1,1,A
7166261,7,F,3,4,0,Parda,SP,1,1,391.4,443.4,498.8,528.4,txt conforme requisitos,400,1,0,0,A


In [70]:
# Média dos estrangeiros na prova de redação
media_red_estrangeiro = enem_2013_df[enem_2013_df['NACIONALIDADE'] == 3]['NOTA_REDACAO'].mean()
print(media_red_estrangeiro)

386.650790513834


In [71]:
# Média dos brasileiros (nativos e naturalizados) na prova de redação
media_red_brasileiro = enem_2013_df[enem_2013_df['NACIONALIDADE'] != 3]['NOTA_REDACAO'].mean()
print(media_red_brasileiro)

358.86958113848874


In [72]:
# Média dos estrangeiros na prova de LC
media_lc_estrangeiro = enem_2013_df[enem_2013_df['NACIONALIDADE'] == 3]['NOTA_LC'].mean()
print(media_lc_estrangeiro)

515.6878811777077


In [73]:
# Média dos brasileiros (nativos e naturalizados) na prova de LC
media_lc_brasileiro = enem_2013_df[enem_2013_df['NACIONALIDADE'] != 3]['NOTA_LC'].mean()
print(media_lc_brasileiro)

489.9892701166278


### Conclusão

- Estrangeiros tiraram notas maiores que os brasileiros, de forma geral em provas relacionadas à língua portuguesa. Isso pode estar atrelado ao fato de que estrangeiros tem uma preocupação maior com o estudo da língua portuguesa do que os próprios brasileiros, que se sentem mais a vontade por se tratar do seu idioma nativo. No entanto, provavelmente os estrangeiros estudam e se atentam mais à norma culta, o que resulta em maiores notas.
---

### Análise de Status da Redação
Qual o perfil dos candidatos que ferem direitos humanos na redação?

In [74]:
# cria uma variável com todos os dados de quem escreveu redacao ferindo direitos humanos
df_fere_dh = enem_2013_df[enem_2013_df['STATUS_REDACAO'] == 'fere DH'].reset_index(drop=True)

# verifica a média de cada uma das 4 matérios, exceto redação que zerou por ferir direitos humanos
df_media_notas = enem_2013_df[['NOTA_CN', 'NOTA_LC', 'NOTA_CH', 'NOTA_MT']].mean()

# mostra as 4 médias
df_media_notas

NOTA_CN    469.010271
NOTA_LC    490.008716
NOTA_CH    519.916705
NOTA_MT    510.371794
dtype: float64

In [75]:
# verifica a média geral das 4 matérias juntas
df_media_geral = df_media_notas.mean()

# mostra a média geral
df_media_geral

np.float64(497.3268713488901)

In [76]:
# Qual gênero anulou mais redações por ferir DH
df_fere_dh['SEXO'].value_counts()

SEXO
F    90
M    74
Name: count, dtype: int64

In [77]:
# Qual cor/raça anulou mais redações por ferir DH
df_fere_dh['COR'].value_counts()

COR
Parda            70
Branca           62
Preta            25
Amarela           3
Indígena          2
Não delcarado     2
Name: count, dtype: int64

In [78]:
# Qual faixa etária anulou mais redações por ferir DH
df_fere_dh['FAIXA_ETARIA'].value_counts()

FAIXA_ETARIA
11    21
2     20
3     19
4     17
1     16
5     15
6     11
9      8
14     8
12     6
13     5
7      5
8      5
15     4
10     3
16     1
Name: count, dtype: int64

In [79]:
# Qual renda familiar anulou mais redações por ferir DH
df_fere_dh['RENDA_FAMILIAR'].value_counts()

RENDA_FAMILIAR
1     72
2     36
3     20
4      8
5      8
8      5
0      5
6      5
10     2
7      2
14     1
Name: count, dtype: int64

In [80]:
# Qual UF anulou mais redações
df_fere_dh['UF_PROVA'].value_counts()

UF_PROVA
SP    25
BA    21
MA    14
MG    13
GO    10
PA     8
PR     8
RJ     8
RS     7
CE     7
PE     6
MS     5
PB     4
MT     4
AM     3
DF     3
SC     3
RN     3
AC     3
ES     3
AL     2
PI     2
RO     1
SE     1
Name: count, dtype: int64

### Conclusão

- Mulheres cometeram 55% das infrações enquanto homens 45%.
- No quesito cor/raça, pessoas pardas anularam 43%, seguida por pessoas brancas 38% e pretas 15%.
- Entre escolas públicas e privadas a análise ficou prejudicada, porque das 164 ocorrências 131 não tem dados,
80% de missing values.
- Analisando faixa etária, os jovens entre 17 a 18 anos e 26 a 30 foram os campeões. Entre os candidatos de 17 a 18 anos representam 24% das anulações, enquanto que candidatos entre 26 a 30 anos foi de 13%.
- No quesito de UF e região, os estados com maior percentual foram SP (15%), BA(13%) e MA(9%). Por região ficou assim: Nordeste 36,58%, Sudeste 29,87%, Centro 13,41%, Sul 10,97% e Norte 9,14%.
- Considerando faixa de renda familiar, 78% das ocorrências aconteceram nas faixas de renda entre menos de um salário mínimo até dois salários mínimos.
---

### Análise Sexo, Raça, Classe

  - Os gêneros performaram diferentemente em cada área do conhecimento?
  - Qual o impacto da classe social (renda média) nas notas de cada gênero?
  - Como a raça (cor) influência na nota de cada gênero?

In [84]:
# Colunas de notas
colunas=['NOTA_CN', 'NOTA_CH', 'NOTA_LC', 'NOTA_MT','NOTA_RE']

# Dataframe Agrupado por gênero
df_social = enem_2013_df.groupby(['SEXO','COR', 'CLASSE'])[colunas].describe().round(2)
df_sexo = enem_2013_df.groupby('SEXO')[colunas].describe().round(2)

KeyError: 'CLASSE'

In [83]:
# Remoção de Outliers

# Tamanho do dataset antes
df_social.shape

NameError: name 'df_social' is not defined